# Import libraries

In [ ]:
import tweepy
import json

# Read Twitter dev keys

In [ ]:
keys_filename = "../keys.json"
consumer_key, consumer_secret, access_token, access_secret = "", "", "", ""
with open(keys_filename) as api_credentials:
    api_twitter = json.loads(api_credentials.read())
    consumer_key = api_twitter['Twitter']["consumer_key"]
    consumer_secret = api_twitter['Twitter']["consumer_secret"]
    access_token = api_twitter['Twitter']["access_token"]
    access_secret = api_twitter['Twitter']["access_secret"]

# API Setup

In [ ]:
#Login
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
#Create API
api = tweepy.API(auth)

# Experimenting with tweets download

In [ ]:
tweets = tweepy.Cursor(api.search,  
              q = "Assetto Corsa",
              since = "2019-06-08", 
              until = "2019-06-09").items()

## Let's have a look to the download tweets

In [ ]:
for tweet in tweets:
    print(tweet)

In [ ]:
tweets.num_tweets

Things to do from now on:
- Create a function that, given a query, a start date and an end date returne the tweets
- Understand how to filter data: we are not interested in all the infos (I think that language + tweet + account_name + follower_count + language may be enough)
- Trasform everything in a Python script so that it will be easier to launch it on the VM

In [ ]:
for tweet in tweets:
    filtered_tweet = {
        "text": tweet.text,
        "language" : tweet.lang,
        "date"     : tweet.created_at,
        "username" : tweet.user.name,
        "user_followers": tweet.user.followers_count,
        "user_location": tweet.user.location,
        "retweets" : tweet.retweet_count,
        "likes"    : tweet.favorite_count,
    }
    print(filtered_tweet)

# Setup mongo

In [8]:
from pymongo import MongoClient
ip_mongo = "localhost"
mongo_port = 27017

client = MongoClient(ip_mongo, mongo_port)
db = client.twitter
tweet_games = db.game_tweets

In [ ]:
def collect_and_store_tweets(api, query, since_date, until_date, mongo_collection):
    tweets = tweepy.Cursor(api.search,  
              q = query,
              since = since_date, 
              until = until_date).items()
    for tweet in tweets:
        filtered_tweet = {
            "text": tweet.text,
            "language" : tweet.lang,
            "date"     : tweet.created_at,
            "username" : tweet.user.name,
            "user_followers": tweet.user.followers_count,
            "user_location": tweet.user.location,
            "retweets" : tweet.retweet_count,
            "likes"    : tweet.favorite_count,
        }
        mongo_collection.insert_one(filtered_tweet)
        

I have improved the code contained in this section and made it a script

# Testing library contained in scripts

In [2]:
import sys
sys.path.insert(0, '../scripts/') #If this doesn't work try removing the double dots or use an absolute path.

In [3]:
from collect_and_store_tweets import collect_and_store_tweets_from_multiple_queries

In [4]:
list_of_queries = ['Assetto Corsa', "Mario Kart 8", "Dark Souls"]

In [5]:
keys_filename = "../keys.json"
ip_mongo="local"
mongo_port=27017
collect_and_store_tweets_from_multiple_queries(keys_filename=keys_filename,
                                               list_of_queries=list_of_queries,
                                               since_date="2019-06-08",
                                               until_date="2019-06-09",
                                               ip_mongo=ip_mongo,
                                               mongo_port=mongo_port)

Working on: Assetto Corsa
Working on: Mario Kart 8
Working on: Dark Souls


TweepError: Twitter error response: status code = 429

In [9]:
query4res = db.game_tweets.find(
        {"query": "Mario Kart 8"}
    ).sort("user.followers_count", -1)
[x for x in query4res]

[{'_id': ObjectId('5cfd3ebe1382d16ec7b4d5d3'),
  'query': 'Mario Kart 8',
  'text': 'RT @LordVash: in about 15 mins were gonna fire up mario kart 8 deluxe and RACE! dont miss out! https://t.co/rJZnqyzzsP https://t.co/zniNxoc…',
  'language': 'en',
  'date': datetime.datetime(2019, 6, 8, 23, 50, 10),
  'username': '˗ˏˋ  posi  ˊˎ˗',
  'user_followers': 215,
  'user_location': '',
  'retweets': 2,
  'likes': 0},
 {'_id': ObjectId('5cfd3ebe1382d16ec7b4d5d4'),
  'query': 'Mario Kart 8',
  'text': 'in about 15 mins were gonna fire up mario kart 8 deluxe and RACE! dont miss out! https://t.co/rJZnqyzzsP https://t.co/zniNxocN5y',
  'language': 'en',
  'date': datetime.datetime(2019, 6, 8, 23, 46, 56),
  'username': 'Vash',
  'user_followers': 13341,
  'user_location': 'Los Angeles, CA',
  'retweets': 2,
  'likes': 15},
 {'_id': ObjectId('5cfd3ebe1382d16ec7b4d5d5'),
  'query': 'Mario Kart 8',
  'text': 'Come join @BlueFuran, @RKikime, and friends for Mario Kart 8 Deluxe (3234-0024-8875)!\n\nhttp